In [65]:
import pandas as pd

In [66]:
df = pd.read_csv('twitter_training.csv')
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [67]:
df = df.rename(columns={'Positive':'label','im getting on borderlands and i will murder you all ,':'review'})

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   2401         74681 non-null  int64 
 1   Borderlands  74681 non-null  object
 2   label        74681 non-null  object
 3   review       73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [69]:
df.isnull().sum()

2401             0
Borderlands      0
label            0
review         686
dtype: int64

In [70]:
df.dropna(inplace=True)

In [71]:
df.duplicated().sum()

2340

In [72]:
df = df.drop_duplicates()

In [73]:
df = df.drop(columns=['2401','Borderlands'])

In [74]:
df['label'].value_counts()

label
Negative      21698
Positive      19712
Neutral       17708
Irrelevant    12537
Name: count, dtype: int64

In [75]:
df = df[df['label'] != 'Irrelevant']

In [76]:
df.reset_index(inplace=True)

In [77]:
df['review'][3847]

'I refuse to buy books off of amazon. I miss the book store . '

In [78]:
df['review'] = df['review'].str.lower()

In [79]:
import re
def remove_specific_characters(text):
    pattern = r'[<\[{\\}"\'\(\)@!-.*?\]>]'
    return re.sub(pattern, '', text)

In [80]:
df.drop(columns='index',inplace=True)

In [81]:
df.iloc[:,1] = df['review'].apply(remove_specific_characters)

In [92]:
from nltk.corpus import stopwords
r_stopwords = stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[item for item in x.split() if item not in r_stopwords]).apply(lambda x:' '.join(x))

In [93]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [96]:
x = df.iloc[:100,1]

In [97]:
y = df.iloc[:100,0]

In [103]:
tokeniz_x = tokenizer(x.to_list(),truncation=True,padding=True,return_tensors='pt')

In [105]:
outputs = model(**tokeniz_x)

In [106]:
import torch
logists = outputs.logits
probabilities = torch.softmax(logists,dim=1)
predicted_labels = torch.argmax(logists,dim=1)

In [107]:
from sklearn.metrics import accuracy_score,multilabel_confusion_matrix

In [129]:
y_test = pd.get_dummies(y)

In [130]:
y_pred = pd.get_dummies(predicted_labels)

In [131]:
print(accuracy_score(y_test,y_pred))

0.33


In [132]:
print(multilabel_confusion_matrix(y_test,y_pred))

[[[65  8]
  [27  0]]

 [[61  6]
  [32  1]]

 [[ 7 53]
  [ 8 32]]]
